# Visualization

Work in progress ...

The goal is to visualize g-code path in the jupyter notebook.

## Setup

In [ ]:
import coursierapi._
interp.repositories() ++= Seq(MavenRepository.of("https://github.com/dzufferey/my_mvn_repo/raw/master/repository"))

In [ ]:
import $ivy.`io.github.dzufferey::libgcode:0.1-SNAPSHOT`
import $ivy.`com.lihaoyi::scalatags:0.8.2`

## Declarations

The plan is to use [X3DOM](https://www.x3dom.org/) for the vizualization.
For rapid prototyping is quite good.

To make the code readable, first the tags and attributes of the subset of X3Dom which will be used.

In [ ]:
import scalatags.Text.all._
import scalatags.stylesheet._
import almond.display.{Html, Text}

object X3D {
    
    //tags
    lazy val x3d = tag("x3d")
    lazy val scene = tag("scene")
    lazy val viewpoint = tag("viewpoint")
    lazy val navigationInfo = tag("navigationinfo")
    lazy val transform = tag("transform")
    lazy val group = tag("group")
    lazy val billboard = tag("billboard")
    lazy val shape = tag("shape")
    lazy val lineSet = tag("lineset")
    lazy val indexedLineSet = tag("indexedlineset")
    lazy val indexedFaceSet = tag("indexedfaceset")
    lazy val plane = tag("plane")
    lazy val sphere = tag("sphere")
    lazy val cone = tag("cone")
    lazy val text = tag("text")
    lazy val fontStyle = tag("fontstyle")
    lazy val coordinate = tag("coordinate")
    lazy val appearance = tag("appearance")
    lazy val material = tag("material")
    lazy val depthMode = tag("depthMode")
    lazy val lineProperties = tag("lineproperties")
    
    //attributes
    lazy val use = attr("use")
    lazy val defn = attr("def")
    lazy val position = attr("position")
    lazy val orientation = attr("orientation")
    lazy val typeParams = attr("typeparams")
    lazy val rotation = attr("rotation")
    lazy val translation = attr("translation")
    lazy val size = attr("size")
    lazy val radius = attr("radius")
    lazy val primType = attr("primtype")
    lazy val subdivision = attr("subdivision")
    lazy val vertexCount = attr("vertexcount")
    lazy val coordIndex = attr("coordindex")
    lazy val index = attr("index")
    lazy val point = attr("point")
    lazy val solid = attr("solid")
    lazy val string = attr("string")
    lazy val colorPerVertex = attr("colorpervertex")
    lazy val lit = attr("lit")
    lazy val color = attr("color")
    lazy val emissiveColor = attr("emissivecolor")
    lazy val diffuseColor = attr("diffusecolor")
    lazy val specularColor = attr("specularcolor")
    lazy val ambientIntensity = attr("ambientintensity")
    lazy val lineWidthScaleFactor = attr("linewidthscalefactor")
    lazy val readOnly = attr("readonly")
    
    val defaultRotation = "1 0 0 -1.570796326795" // z axis up
    
    //stylsheet
    object Simple extends StyleSheet {
        initStyleSheet()
        val mainViewer = cls(
            backgroundColor := "rgba(128, 128, 196, 0.4)",
            borderStyle := "solid"
        )
    }
    
    def grid(_size: Int, step: Int) = {
        val grey = appearance( material( diffuseColor := "0.0 0.0 0.0", specularColor := "0.0 0.0 0.0", emissiveColor := "0.3 0.3 0.3" ) )
        val sSize = s"${2*_size} ${2*_size}"
        val sStep = s"${2*_size/step} ${2*_size/step}"
        val sBorder = s"-${_size} -${_size} 0.0 , -${_size} ${_size} 0.0 , ${_size} ${_size} 0.0, ${_size} -${_size} 0.0"
        group(id := "grid")(
            shape(
                plane(id := "innerGrid", solid := "false", size := sSize, primType := "LINES", subdivision := sStep ),
                grey
            ),
            shape(
                indexedLineSet(coordIndex := "0 1 2 3 0", colorPerVertex := "false", lit := "false")(
                    coordinate(point := sBorder)
                ),
                grey
            )
        )
    }
    
    def axes(length: Int, step: Int) = {
        def color(c: String) = {
            appearance(
                material( diffuseColor := "0 0 0", specularColor := "0 0 0", emissiveColor := c ),
                lineProperties( lineWidthScaleFactor := "1.1" )
            )
        }
        group(id := "axes")(
            shape( // X
                lineSet( vertexCount := "2" ) (
                    coordinate( point := s"0 0 0.001, $length 0 0.001")
                ),
                color("1 0 0")
            ),
            shape( // Y
                lineSet( vertexCount := "2" ) (
                    coordinate( point := s"0 0 0.001, 0 $length 0.001")
                ),
                color("0 1 0")
            ),
            shape( // Z
                lineSet( vertexCount := "2" ) (
                    coordinate( point := s"0 0 0, 0 0 $length")
                ),
                color("0 0 1")
            ),
            for (z <- 0 to length by step) yield {
                shape(
                    lineSet( vertexCount := "2" ) (
                        coordinate( point := s"0 0 $z, $step 0 $z")
                    ),
                    color("0 0 1")
                )
            }
        )
    }
    
    def viewer(content: Modifier) = {
        div(backgroundColor := "rgba(128, 128, 196, 0.4)",
            borderStyle := "solid",
            script( tpe := "text/javascript", src := "https://www.x3dom.org/download/x3dom.js" ),
            link( rel := "stylesheet", tpe := "text/css", href := "https://www.x3dom.org/download/x3dom.css" ),
            tag("style")(Simple.styleSheetText),
            x3d(Simple.mainViewer)(
                scene(
                    viewpoint( id:= "viewPoint", position := "5.53912 7.69774 6.54642" , orientation := "-0.69862 0.66817 0.25590 1.00294" ),
                    navigationInfo( id := "navi", tpe := "\"TURNTABLE\" \"ANY\"", typeParams := "-0.4, 60, 0.5, 1.55" ),
                    transform(rotation := defaultRotation)(
                        grid(10, 1),
                        axes(100, 1),
                        content
                    )
                )
            )
        ).render
    }
    
    //some helpers to create elements
    def display(content: Modifier) =  Html(viewer(content))
    
}

A simple test

In [ ]:
X3D.display(
    X3D.shape(
        X3D.sphere(X3D.radius := "0.2"),
        X3D.material( X3D.emissiveColor := "0.8 0.8 0.8" )
    )
)

Now that there is a basic viewer, we need to turn "plot" g-code.
For that the simplest is to extend `AbstractMachine` so that it produces lines for the different motions.

In [ ]:
// TODO ...